# Hyperparameter Tuning using HyperDrive

In [8]:
from azureml.core import Workspace, Environment, Experiment, Model
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import BayesianParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice
from azureml.train.hyperdrive.parameter_expressions import uniform
import os
import pandas as pd
ws = Workspace.get(name="quick-starts-ws-130382")
exp = Experiment(workspace=ws, name="lol_match_predictor")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
# Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

cluster_name = "cluster"

# Create a provisioning configuration using the Standard_D2_V2 VM and capping it at 4 nodes.
compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2", max_nodes=4)
# Create compute cluster.
cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
experiment_name = 'lol_match_predictor'

experiment=Experiment(ws, experiment_name)

NameError: name 'Workspace' is not defined

In [9]:
data = pd.read_csv("OraclesElixir_lol_match_data_teams.csv")

# Show raw data before it is processed.
data.head()

,side,position,player,team,champion,ban1,ban2,ban3,ban4,ban5,...,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,killsat15,assistsat15,deathsat15,opp_killsat15,opp_assistsat15,opp_deathsat15
0,Blue,top,Summit,SANDBOX Gaming,Rumble,LeBlanc,Irelia,Rek'Sai,Yasuo,Renekton,...,118.0,165.0,166.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Blue,jng,OnFleek,SANDBOX Gaming,Elise,LeBlanc,Irelia,Rek'Sai,Yasuo,Renekton,...,98.0,-399.0,150.0,-7.0,0.0,0.0,0.0,1.0,0.0,0.0
2,Blue,mid,FATE,SANDBOX Gaming,Qiyana,LeBlanc,Irelia,Rek'Sai,Yasuo,Renekton,...,140.0,-409.0,-1837.0,-11.0,0.0,0.0,1.0,0.0,1.0,0.0
3,Blue,bot,Leo,SANDBOX Gaming,Miss Fortune,LeBlanc,Irelia,Rek'Sai,Yasuo,Renekton,...,135.0,51.0,-401.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Blue,sup,GorillA,SANDBOX Gaming,Nautilus,LeBlanc,Irelia,Rek'Sai,Yasuo,Renekton,...,28.0,-233.0,257.0,-8.0,0.0,0.0,0.0,0.0,1.0,0.0


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [ ]:
#TODO: Create the different params that you will be using during training
param_sampling = BayesianParameterSampling( {
        "--C": uniform(0.1,10),
        "--max_iter": choice(20,40,60,80,100,150,200,300,400,600,800)
    }
)

if "training" not in os.listdir():
    os.mkdir("./training")

#TODO: Create your estimator and hyperdrive config
estimator = SKLearn(".", compute_target=cpu_cluster, entry_script="train.py")

hyperdrive_run_config = HyperDriveConfig(ps,
                                     estimator=est,
                                     primary_metric_name="Accuracy",
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=100,
                                     max_concurrent_runs=4)






In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = exp.submit(hyperdrive_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(hyperdrive_run).show()

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments'])

In [ ]:
#TODO: Save the best model

joblib.dump(value=best_run.id, filename="model.pkl")

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
model = Model.register(model_path = ".",
                       model_name = "predictor",
                       description = "League of Legends match predictor trained using Azure ML.",
                       workspace = ws)

# Get environment
env = Environment.get(workspade=ws, name="AzureML-AutoML")

# Create inference configuration
inference_config = InferenceConfig(entry_script="score.py", environment=env)

# Set deployment configuration as an Azure Container Instance
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=4)

# Enable logging/application insights
service.update(enable_app_insights=True)

# Service deployment
service = Model.deploy(
    workspace=ws,
    name="predictor",
    models=[model],
    inference_config=inference_config,
    deployment_config=deployment_config
)

service.wait_for_deployment(show_output=True)
print(service.state)


TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
scoring_uri = service.scoring_uri

print("Service State: {}".format(service.state))
print("Scoring URI: {}".format(service.scoring_uri))
print("Swagger URI: {}".format(service.swagger_uri))


In [10]:
import requests
import json

data = {'data': [ -500, -300, 2000, 1600] }

input = json.dumps(data)

headers = {'Content-Type': 'application/json'}

response = requests.post(scoring_uri, input_data, headers=headers)
print(response.text)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
service = Webservice(name='predictor', workspace=ws)
print(service.get_logs())

In [ ]:
service.delete()